# Lab 3: Linear Classification

In [ ]:
import numpy as np

## 1. StandardizeData Function

This function standardizes features by removing the mean and scaling to unit variance. The mean and standard deviation are calculated *only* from the training data (`X_train`) and then applied to all three datasets (`X_train`, `X_val`, `X_test`). This prevents data leakage from the validation and test sets into the training process.

**Standardization Formula:**
Z = (X - mean_train) / (std_train + epsilon)

Where:
- `X` is the data to be standardized (a feature column).
- `mean_train` is the mean of the feature in the training data.
- `std_train` is the standard deviation of the feature in the training data.
- `epsilon` is a small constant (e.g., 1e-8) added to the standard deviation to prevent division by zero in case a feature has zero variance in the training set.

In [ ]:
def StandardizeData(X_train, X_val, X_test):
    """
    Standardizes the training, validation, and test datasets based on the 
    mean and standard deviation of the training data.

    Args:
        X_train (np.ndarray): Training data features (samples x features).
        X_val (np.ndarray): Validation data features (samples x features).
        X_test (np.ndarray): Test data features (samples x features).

    Returns:
        tuple: (X_train_standardized, X_val_standardized, X_test_standardized)
               The standardized versions of the input datasets.
    """
    if X_train.ndim == 1:
        X_train = X_train.reshape(-1, 1)
        if X_val.ndim == 1: X_val = X_val.reshape(-1, 1)
        if X_test.ndim == 1: X_test = X_test.reshape(-1, 1)
            
    mean_train = np.mean(X_train, axis=0)
    std_train = np.std(X_train, axis=0)
    epsilon = 1e-8
    
    X_train_standardized = (X_train - mean_train) / (std_train + epsilon)
    
    if X_val.shape[1] == mean_train.shape[0]:
        X_val_standardized = (X_val - mean_train) / (std_train + epsilon)
    else:
        print(f"Warning: X_val feature count ({X_val.shape[1]}) differs from X_train ({mean_train.shape[0]}). Returning original X_val.")
        X_val_standardized = X_val
    
    if X_test.shape[1] == mean_train.shape[0]:
        X_test_standardized = (X_test - mean_train) / (std_train + epsilon)
    else:
        print(f"Warning: X_test feature count ({X_test.shape[1]}) differs from X_train ({mean_train.shape[0]}). Returning original X_test.")
        X_test_standardized = X_test
    
    return X_train_standardized, X_val_standardized, X_test_standardized

### Example Usage for StandardizeData

In [ ]:
X_train_sample_sd = np.array([[1, 10, 100], [2, 20, 200], [3, 30, 300], [4, 40, 400]], dtype=float)
X_val_sample_sd = np.array([[5, 50, 500], [6, 60, 600]], dtype=float)
X_test_sample_sd = np.array([[0, 0, 0], [7, 70, 700]], dtype=float)
X_train_std_sd, X_val_std_sd, X_test_std_sd = StandardizeData(X_train_sample_sd.copy(), X_val_sample_sd.copy(), X_test_sample_sd.copy())

## 2. Perceptron Model

This section defines the `Perceptron` class for binary classification.

### 2.1. Perceptron Class Definition

In [ ]:
class Perceptron(object):
    """Perceptron classifier.

    Parameters
    ------------
    X : np.ndarray (for shape determination in __init__)
      Input data used to determine the number of features and initialize weights.
      This is typically the training data X.
    n_iter : int
      Number of passes over the training dataset (epochs). Default is 1.

    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting. The first element is the bias term (w_0),
      and the rest are the weights for each feature (w_1, w_2, ...).
    errors_ : list
      Number of misclassifications (updates) in each epoch. Populated by 'fit'.
    n_iter : int
      Stores the number of iterations specified for training.
      
    Methods
    -------
    fit(X, y)
      Fits the training data to learn model weights.
    linear_combination(X)
      Calculates the net input (weighted sum + bias).
    predict(X)
      Returns class labels after applying a step function to the net input.
    """
    def __init__(self, X, n_iter=1):
        if not isinstance(X, np.ndarray):
            raise ValueError("Input X must be a NumPy array for Perceptron initialization.")
        if X.ndim == 1:
            X_used_for_shape = X.reshape(1, -1)
        else:
            X_used_for_shape = X
            
        self.w_ = np.zeros(1 + X_used_for_shape.shape[1]) # +1 for the bias term w_0
        self.errors_ = []
        self.n_iter = n_iter

    def linear_combination(self, X):
        """Calculate net input (weighted sum).
        """
        if X.ndim == 1:
            X_proc = X.reshape(1, -1)
        else:
            X_proc = X
            
        if X_proc.shape[1] != (len(self.w_) - 1):
            raise ValueError(f"Number of features in X ({X_proc.shape[1]}) does not match number of weights ({len(self.w_) - 1}).")
            
        return np.dot(X_proc, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step function.
        """
        net_input = self.linear_combination(X)
        predictions = np.where(net_input >= 0, 1, 0)
        return predictions

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values (0 or 1).

        Returns
        -------
        self : object
        """
        # print(f"Initial weights: {self.w_}") # Optional: for debugging
        self.errors_ = [] 

        for epoch_num in range(self.n_iter):
            errors_in_epoch = 0
            for xi, target_label in zip(X, y):
                prediction = self.predict(xi) 
                
                if isinstance(prediction, np.ndarray) and prediction.ndim > 0:
                    prediction_scalar = prediction[0]
                else:
                    prediction_scalar = prediction

                update = target_label - prediction_scalar
                
                if update != 0:
                    self.w_[1:] += update * xi
                    self.w_[0] += update 
                    errors_in_epoch += 1
            
            self.errors_.append(errors_in_epoch)
            # Optional: print epoch-wise updates for debugging
            # print(f"Errors in epoch {epoch_num + 1}: {errors_in_epoch}")
            # print(f"Updated weights after epoch {epoch_num + 1}: {self.w_}")
            
        return self

### Example Usage for Perceptron (Initialization, Linear Combination, Prediction, and Fit)

In [ ]:
X_sample_p_ex = np.array([[2.0, 3.0], [0.5, 1.5], [4.0, 0.0], [1.0, 1.0], [3.0, 2.0]])
y_sample_p_ex = np.array([1, 0, 1, 0, 1]) 
perceptron_ex_fit = Perceptron(X=X_sample_p_ex, n_iter=3) 
# print(f"Initial weights: {perceptron_ex_fit.w_}")
perceptron_ex_fit.fit(X_sample_p_ex, y_sample_p_ex)
# print(f"Errors per epoch: {perceptron_ex_fit.errors_}")
# print(f"Final weights: {perceptron_ex_fit.w_}")
# predictions_after_fit_ex = perceptron_ex_fit.predict(X_sample_p_ex)
# print(f"Predictions on X_sample_p_ex after fitting: {predictions_after_fit_ex}")

## 3. Fisher's Linear Discriminant Function

This function computes the Fisher's Linear Discriminant vector `w` which optimally separates two classes.

**Methodology:**
1.  **Compute Mean Vectors**: Calculate the mean vector for each class (0 and 1).
    - `m0 = mean(X[y == 0])`
    - `m1 = mean(X[y == 1])`
2.  **Compute Within-Class Scatter Matrices (S0, S1)**:
    - For each class, calculate its scatter matrix. For class 0:
      `s0_scatter = sum((x - m0)(x - m0)^T)` for all `x` in class 0.
    - Similarly for `s1_scatter` for class 1 using `m1`.
3.  **Compute Total Within-Class Scatter Matrix (S_W)**:
    - `S_W = s0_scatter + s1_scatter`
4.  **Compute Between-Class Scatter Matrix (S_B)**:
    - `S_B = (m1 - m0)(m1 - m0)^T`
    - (Note: `S_B` itself is not directly used to find `w` in the `inv(S_W) * (m1-m0)` formulation, but it's part of Fisher's criterion J(w) = (w^T S_B w) / (w^T S_W w). The direction that maximizes this is proportional to `inv(S_W) * (m1-m0)`).
5.  **Compute Discriminant Vector (w)**:
    - `w = inv(S_W) @ (m1 - m0)`
    - If `S_W` is singular, its pseudo-inverse `pinv(S_W)` is used.
6.  **Normalize Discriminant Vector**: 
    - `w = w / ||w||` (Euclidean norm)
    This makes the vector a unit vector, which is common practice but doesn't change its direction.

In [ ]:
def fisher_discriminant(X, y):
    """
    Computes Fisher's Linear Discriminant vector w.

    Args:
        X (np.ndarray): Standardized input data (samples x features).
        y (np.ndarray): Binary labels (0 or 1) for each sample.

    Returns:
        np.ndarray: Normalized Fisher's Linear Discriminant vector w.
                    Returns None if computation fails (e.g., singular S_W and pinv also fails, or class has no samples).
    """
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    
    num_features = X.shape[1]

    # Separate data by class
    X0 = X[y == 0]
    X1 = X[y == 1]

    if X0.size == 0 or X1.size == 0:
        print("Warning: One or both classes have no samples. Cannot compute Fisher Discriminant.")
        return None

    # Compute mean vectors
    m0 = np.mean(X0, axis=0)
    m1 = np.mean(X1, axis=0)

    # Compute within-class scatter matrices (S0 and S1)
    s0_scatter = np.zeros((num_features, num_features))
    for row in X0:
        diff = (row - m0).reshape(num_features, 1)
        s0_scatter += np.dot(diff, diff.T)
    
    s1_scatter = np.zeros((num_features, num_features))
    for row in X1:
        diff = (row - m1).reshape(num_features, 1)
        s1_scatter += np.dot(diff, diff.T)

    # Total within-class scatter matrix S_W
    S_W = s0_scatter + s1_scatter

    # Compute discriminant vector w
    try:
        S_W_inv = np.linalg.inv(S_W)
    except np.linalg.LinAlgError:
        print("Warning: S_W is singular, using pseudo-inverse.")
        try:
            S_W_inv = np.linalg.pinv(S_W)
        except np.linalg.LinAlgError:
            print("Error: Pseudo-inverse of S_W also failed. Cannot compute w.")
            return None
            
    w = S_W_inv @ (m1 - m0)
    
    # Normalize discriminant vector w
    norm_w = np.linalg.norm(w)
    if norm_w == 0:
        print("Warning: Norm of w is zero. Cannot normalize.")
        return w # Or None, or handle as appropriate
    w_normalized = w / norm_w
    
    return w_normalized

### Example Usage for fisher_discriminant

In [ ]:
# Sample standardized data for Fisher's LDA
X_exp_fisher = np.array([
    [-1, -1], [-1.5, -0.5], [-0.5, -1.5], [-0.8, -0.8], # Class 0
    [ 1,  1], [ 1.5,  0.5], [ 0.5,  1.5], [ 0.8,  0.8]  # Class 1
])
y_exp_fisher = np.array([0, 0, 0, 0, 1, 1, 1, 1])

# print("Sample X for Fisher Discriminant:\n", X_exp_fisher)
# print("Sample y for Fisher Discriminant:\n", y_exp_fisher)

w_fisher_example = fisher_discriminant(X_exp_fisher, y_exp_fisher)

if w_fisher_example is not None:
    pass # print("\nCalculated Fisher's Discriminant vector (w):", w_fisher_example)


## 4. Fisher's LDA Decision Boundary Calculation

Once data is projected onto the 1D space defined by Fisher's discriminant vector `w`, a decision boundary is needed to classify new points. This boundary is typically chosen as the midpoint between the means of the two classes in this projected 1D space.

**Methodology:**
1.  **Project Data**: The training data `X_train` is projected onto `w`: `X_train_lda = X_train @ w`.
2.  **Calculate Means of Projected Classes**: 
    - `mean_class_0_lda = mean(X_train_lda[y_train == 0])`
    - `mean_class_1_lda = mean(X_train_lda[y_train == 1])`
3.  **Compute Decision Boundary**: 
    - `decision_boundary = (mean_class_0_lda + mean_class_1_lda) / 2.0`
    This boundary value is a scalar in the 1D projected space.

In [ ]:
def boundary_calculation(X_train_lda, y_train):
    """
    Calculates the decision boundary for 1D LDA-projected data.

    Args:
        X_train_lda (np.ndarray): Training data projected onto the LDA vector (1D array).
        y_train (np.ndarray): Binary labels (0 or 1) for the training data.

    Returns:
        float: The calculated decision boundary.
               Returns np.nan if one or both classes are empty in the training data.
    """
    if X_train_lda.ndim != 1:
        if X_train_lda.shape[1] == 1:
            X_train_lda = X_train_lda.flatten()
        else:
            raise ValueError("X_train_lda must be a 1D array or a 2D array with one column.")

    class_0_projected = X_train_lda[y_train == 0]
    class_1_projected = X_train_lda[y_train == 1]

    if class_0_projected.size == 0:
        print("Warning: Class 0 has no samples in X_train_lda. Cannot compute its mean.")
        mean_class_0_lda = np.nan
    else:
        mean_class_0_lda = np.mean(class_0_projected)
        
    if class_1_projected.size == 0:
        print("Warning: Class 1 has no samples in X_train_lda. Cannot compute its mean.")
        mean_class_1_lda = np.nan
    else:
        mean_class_1_lda = np.mean(class_1_projected)

    if np.isnan(mean_class_0_lda) or np.isnan(mean_class_1_lda):
        print("Warning: Mean for one or both classes is NaN. Boundary calculation failed.")
        return np.nan
        
    decision_boundary = (mean_class_0_lda + mean_class_1_lda) / 2.0
    
    return decision_boundary

### Example Usage for boundary_calculation

In [ ]:
if 'w_fisher_example' in globals() and w_fisher_example is not None:
    if X_exp_fisher.shape[1] == len(w_fisher_example):
        X_exp_lda = X_exp_fisher @ w_fisher_example 
        # print("Projected X_exp_fisher (X_exp_lda):\n", X_exp_lda)
        boundary_example = boundary_calculation(X_exp_lda, y_exp_fisher)
        # print(f"\nCalculated Decision Boundary: {boundary_example:.4f}")
    # else: print("Skipping boundary_calculation example due to shape mismatch.")
else:
    print("Skipping boundary_calculation example as w_fisher_example is not available.")

## 5. LDA Classifier Function

This function uses Fisher's LDA to classify test data. It first computes the discriminant vector `W` from the training data, projects both training and test data onto `W`, calculates a decision boundary from the projected training data, and then classifies the projected test data based on this boundary.

In [ ]:
def lda_classifier(X_train, y_train, X_test):
    """
    Classifies test data using Fisher's Linear Discriminant Analysis.

    Args:
        X_train (np.ndarray): Standardized training data (samples x features).
        y_train (np.ndarray): Binary labels (0 or 1) for X_train.
        X_test (np.ndarray): Standardized test data (samples x features).

    Returns:
        np.ndarray: Predicted labels (0 or 1) for X_test. 
                    Returns an empty array if classification cannot proceed.
    """
    # Ensure inputs are 2D, this is handled by fisher_discriminant for X_train
    # but good to ensure for X_test as well for projection.
    if X_train.ndim == 1:
        X_train = X_train.reshape(-1,1)
    if X_test.ndim == 1:
        X_test = X_test.reshape(-1,1)
    
    # 1. Get Discriminant Vector W
    W = fisher_discriminant(X_train, y_train)
    if W is None:
        print("LDA Classifier Error: Could not compute discriminant vector W.")
        return np.array([]) # Return empty array for predictions

    # 2. Project Data
    # Ensure W is 1D for dot product with X_train (N, D) @ W (D,) -> (N,)
    if W.ndim > 1 and W.shape[1] == 1:
        W = W.flatten() # Convert column vector to 1D array if necessary
    elif W.ndim > 1:
        print(f"LDA Classifier Error: Discriminant vector W has unexpected shape {W.shape}.")
        return np.array([])
        
    X_train_lda = X_train @ W
    X_test_lda = X_test @ W

    # 3. Calculate Decision Boundary
    decision_boundary = boundary_calculation(X_train_lda, y_train)
    if np.isnan(decision_boundary):
        print("LDA Classifier Error: Could not compute decision boundary (resulted in NaN).")
        return np.array([]) # Return empty array for predictions

    # 4. Classify Test Data
    # If w = inv(Sw)(m1 - m0), then m1_projected > m0_projected (usually, if w points towards class 1)
    # So, if X_test_lda >= decision_boundary, it's closer to mean of class 1.
    # We need to determine which class corresponds to larger projected values.
    # Let's check means on projected training data.
    mean_class_0_lda = np.mean(X_train_lda[y_train == 0])
    mean_class_1_lda = np.mean(X_train_lda[y_train == 1])

    if mean_class_1_lda >= mean_class_0_lda:
        # Class 1 has larger (or equal) projected values
        y_pred = np.where(X_test_lda >= decision_boundary, 1, 0)
    else:
        # Class 0 has larger projected values (w might be pointing in the opposite direction)
        y_pred = np.where(X_test_lda < decision_boundary, 1, 0) # Note: < for class 1 if m1 < m0
        # Or, more consistently, predict class 0 if X_test_lda >= boundary (closer to m0_proj)
        # y_pred = np.where(X_test_lda >= decision_boundary, 0, 1) # This line is more consistent with m0 > m1
        # Let's stick to the rule: if projected value > boundary, it's class 1, if W was defined as m1-m0. 
        # The boundary is midpoint. The direction of W (m1-m0) implies m1_proj > m0_proj.
        # So, X_test_lda >= decision_boundary should generally correspond to class 1.
        # The check above handles if m1_proj < m0_proj (e.g. due to S_W effects or if w was -(m1-m0))
        # The most robust way is to compare a test point's projection to the projected means.
        # However, the prompt implies a simple threshold comparison.
        # The current `np.where(X_test_lda >= decision_boundary, 1, 0)` is standard if m1_proj > m0_proj.
        # If m0_proj > m1_proj, then this rule effectively assigns to class 0 for values > boundary.
        # Let's make it explicit: assign to class whose projected mean is further in that direction.
        if mean_class_1_lda > mean_class_0_lda:
            y_pred = np.where(X_test_lda >= decision_boundary, 1, 0)
        else: # mean_class_0_lda > mean_class_1_lda (or equal, covered by first case)
            y_pred = np.where(X_test_lda >= decision_boundary, 0, 1) # If value > boundary, it's closer to m0

    return y_pred

### Example Usage for lda_classifier

In [ ]:
# Using X_exp_fisher and y_exp_fisher from the fisher_discriminant example as training data
X_train_lda_example = X_exp_fisher
y_train_lda_example = y_exp_fisher

# Create sample test data (standardized)
X_test_lda_example = np.array([
    [-2, -2],   # Expected class 0
    [ 2,  2],   # Expected class 1
    [ 0,  0],   # Expected boundary case, depends on tie-breaking or exact boundary
    [-0.5, 1],  # Mixed, depends on projection
    [ 1, -0.5]   # Mixed, depends on projection
])

print("LDA Classifier Example:")
print("X_train:\n", X_train_lda_example)
print("y_train:\n", y_train_lda_example)
print("X_test:\n", X_test_lda_example)

y_predictions_lda = lda_classifier(X_train_lda_example, y_train_lda_example, X_test_lda_example)

if y_predictions_lda is not None and y_predictions_lda.size > 0:
    print("\nPredicted labels for X_test:", y_predictions_lda)
    # For X_exp_fisher, w_fisher_example pointed towards class 1 (larger projected values for class 1).
    # Boundary was near 0. 
    # Test point [-2,-2] projected by w ~[0.7,0.7] is ~ -2.8 -> class 0
    # Test point [2,2] projected by w ~[0.7,0.7] is ~ 2.8 -> class 1
    # Test point [0,0] projected is 0 -> class 1 (if boundary is ~0 and rule is >= boundary -> 1)
    # Expected (approximate): [0, 1, 1, ?, ?] - last two depend on projection relative to boundary
else:
    print("LDA classification failed or returned no predictions.")

# Example with 1D data
X_train_1d_lda = np.array([-1, -2, -0.5, 1, 2, 0.5]).reshape(-1, 1)
y_train_1d_lda = np.array([0, 0, 0, 1, 1, 1])
X_test_1d_lda = np.array([-3, 0, 3]).reshape(-1,1)

print("\nLDA Classifier Example (1D data):")
y_preds_1d_lda = lda_classifier(X_train_1d_lda, y_train_1d_lda, X_test_1d_lda)
if y_preds_1d_lda is not None and y_preds_1d_lda.size > 0:
    print("Predicted labels for 1D X_test:", y_preds_1d_lda)
    # m0 ~ -1.16, m1 ~ 1.16. Boundary ~0. W for 1D is [1] or [-1]. Let's say W=[1].
    # X_test_lda = [-3,0,3]
    # Predictions: [0,1,1] (if boundary=0, rule is >=0 -> 1)
else:
    print("LDA classification for 1D data failed or returned no predictions.")